In [ ]:
import numpy as np

In [ ]:
N = 32
x = np.ones( (N,N) ) * -1
y, z = np.linspace(-1,1,N),np.linspace(-1,1,N)
ys, zs = np.meshgrid( y,z)

x0s = np.concatenate( [x[...,None],ys[...,None],zs[...,None]], axis=2).reshape(N**2, 3)

In [ ]:
celdas = {}
# almaceno (i,j,k) que son los indices de cada celda

In [ ]:
def sdf( x ):
    return np.linalg.norm( x, axis=1 )[...,None] - 0.5

In [ ]:
def indices_celdas( s1 ):
    offsets = np.array([
        [0,0,0],
        [0,-1,0],
        [0,0,1],
        [0,-1,1]
    ])

    return np.repeat(s1, 4, axis=0) + np.tile( offsets, (len(s1),1))

In [ ]:
def actualizar_dict( d, idxs, vertexs, normals ):
    for i,v,n in zip(idxs,vertexs,normals):
        if tuple(i) in d:
            d[tuple(i)]['vertexs'].append( v )
            d[tuple(i)]['normals'].append( n )
        else:
            d[tuple(i)] = { 'vertexs': [ v ], 'normals': [n] }            
    

In [ ]:
indices = np.concatenate( np.array(np.meshgrid( np.arange(N), np.arange(N)))[...,None], axis=2 )
distancias = np.abs( sdf(x0s) ).reshape((N,N))
jks = indices[ (distancias < 1) ]

jks

In [ ]:
vivos = np.ones( N**2,dtype=bool)
indices = np.concatenate( np.array(np.meshgrid( np.arange(N), np.arange(N)))[...,None], axis=2 )
EPS = 1e-5

while np.sum(vivos) > 0:
    print(np.sum(vivos))
    distancias = np.abs( sdf(x0s) )
    x0s[...,0][...,None] += distancias
    vivos *= x0s[..., 0] < 1

    mascara_distancias = (distancias.flatten() < EPS) * vivos

    impactos = x0s[ mascara_distancias ]
    jks = indices[ (mascara_distancias).reshape((N,N)) ]
    i = np.floor(( impactos[...,0] + 1) / 2 * N).astype(np.uint16)

    aristas_impactadas = np.concatenate( [i[...,None] , jks ], axis=-1)
    celdas_impactadas = indices_celdas( aristas_impactadas )
    
    actualizar_dict( 
        celdas, 
        celdas_impactadas, 
        np.repeat(impactos, 4, axis=0), 
        np.repeat( impactos / np.linalg.norm(impactos, axis=-1)[...,None],4, axis=0) 
    )

    
    x0s[mascara_distancias, 0] = np.ones( np.sum(mascara_distancias) ) * -1 + (i+1.01) * (2 / N)
    
    

In [ ]:
a = np.array( [[0,1],[1,2],[2,3]])

a[...,0][...,None]

In [ ]:
celdas

In [1]:
from mc import generateSurface, sdf, cubeVertexs, cubeCenterVertex, tracedMarchingCubes, genGrid
import meshplot as mp
import numpy as np

In [2]:
d, ev, p = cubeVertexs( 8, sdf, surfaceThresh=1e-5 )
cubeCenterVertex( d, ev )

v,t = generateSurface( d, p )

In [3]:
edgeVertexs = [ ev[ i['edgeVertexIndexs']] for i in d.values() ]
vertexs = np.array([ i['vertex'] for i in d.values() ] )

In [ ]:
len(edgeVertexs)

In [4]:
I = 27
p = mp.plot(edgeVertexs[I], shading={'point_size':0.17}, return_plot=True)
#p.add_lines( vertexs, normals )
p.add_points( ev, shading={'point_color': 'blue', 'point_size':0.1})
p.add_points( np.array( [vertexs[I]] ), shading={'point_color':'orange', 'point_size':0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.178575…

2

In [11]:
v,t = tracedMarchingCubes( 32 )
mp.plot(v,t)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.980232…

In [ ]:
grid = np.concatenate( [ genGrid( 8, i ) for i in range(3) ], axis=0)

p = mp.plot(grid, shading={'point_size':0.2}, return_plot=True)
p.add_points( grid[64 + 8: 64 + 12, :], shading={'point_color':'blue','point_size':0.2})
p.add_lines( np.zeros((3,3)), np.array([[1,0,0],[0,1,0],[0,0,1]]))


In [ ]:
N = 8
x,y=np.meshgrid( np.arange(N),np.arange(N), indexing='ij' )
np.concatenate( [x[...,None], y[...,None]], axis=2 )